# COMPLETE TESS LC INFORMATION FOR CARM_VIS OBJECTS

In [1]:
# NOTA IMPORTANTE - CÓMO IMPLEMENTAR LO DEL "TIMEOUT".
# https://stackoverflow.com/questions/25027122/break-the-function-after-certain-time
# NO SIRVE - SOLO SIRVE PARA ENTORNOS UNIX

In this notebook we complete the TESS information for CARM_VIS objects, trying to use the values already calculated for the previous batch of GTO objects, so as not to repeat the TESS GLS calculations again, as they computation cost is high.

**Calculation conditions:**

The stars currently under analysis are in the $He_{3}$ instability band, so they are expected to have effective temperatures in the range $T_{eff}\in[3300, 4300] K$, $\log g\in[4.5, 5.1]$, and masses in the range $M_{star}\in[0.20, 0.60] M_{Sun}$.

According to _Table 2_ in _The theoretical instability strip of M dwarf stars (Rodríguez-López, C., et al. 2014, MNRAS, 438, 2371)_ these stars have typical periods of $20 min$ to $3 h$, so we will set the limits of the periodogram frequencies for periods between $10 min$ and $5 h$. In days, this corresponds to a range of $P\in[0.006944, 0.208333] d$ or, equivalently, frequencies in the range $f\in[144, 4.8] d^{-1}$.

We will use the _Generalized Lomb Scargle Periodogram_ method, as described in [Zechmeister and Kürster, 2009](https://www.aanda.org/articles/aa/full_html/2009/11/aa11296-08/aa11296-08.html) and implemented by GitHub repository [mzechmeister/GLS](https://github.com/mzechmeister/GLS), using the default _ZK_ normalization.

## Modules and configuration

### Modules

In [2]:
# Modules import:
#from collections import OrderedDict
import pandas as pd
import numpy as np
import time

# https://github.com/mzechmeister/GLS
from gls import Gls

from astropy.table import Table, QTable
#from astropy.timeseries import TimeSeries
from astropy import units as u
from astropy.io import fits
#from astropy.time import Time

import lightkurve as lk

#%matplotlib inline
import matplotlib.pyplot as plt

from pylab import rcParams
rcParams['figure.figsize'] = 11, 11

#import seaborn as sns
#sns.set_style("white", {'figure.figsize':(15,10)})
#sns.set_style("whitegrid")
#sns.set(rc={'figure.figsize':(15,8)})

### Configuration

In [3]:
# Configuration:
# Files and folders:
GTO_FILE = "../data/CARM_VIS_objects_with_PG.csv"
REF_GTO_FILE = "../data/GTO_objects_with_PG.csv"
IN_LC_FOLDER = "../data/CARMENES_GTO_TESS_lc/lightcurves/"
OUT_IMG_FOLDER = "../data/CARMENES_GTO_TESS_PGs/figures/"
OUT_PROCESSED_FOLDER = "../data/CARMENES_GTO_TESS_PGs/"
TESS_EXCEPTION_FILE = "../data/CARM_VIS_TESS_Exceptions.csv"
#OUT_GTO_FILE = "../data/GTO_objects_withRVPG.csv"
#OUT_PG_FILE = "../data/GTO_PGs.csv"
#OUT_ERROR_FILE = "../data/GTO_PG_ERRORs.csv"

# Periodogram constants:
FBEG = 4.8
FEND = 144
PBEG = None # Default
PEND = None # Default
OFAC = 10 # Default
HIFAC = 1 # Default
FREQ = np.linspace(start=FBEG, stop=FEND, num=1000001) # 1,000,001 points between 4.8 and 144 d^(-1)
# Must be compatible with FBEG, FEND values
NORM = "ZK" # Default
LS = False # Default
FAST = False # Default
VERBOSE = False # Default
FAP_LEVELS_PLOT = [0.01, 0.05, 0.10] # FAP reference levels to plot

# Initial timeout value (to prevent a stuck file to interrupt the whole process)
#INITIAL_TIMEOUT = 600 # Seconds


In [4]:
def tess_lc_load(filename: str):
    '''Load the TESS LC file and returns a lightcurve object with just the three relevant columns
    Note: it seems to be necessary to drop 'nan' values for GLS to work properly.'''
    lc_lk = lk.read(filename).remove_nans()
    return lc_lk

In [5]:
def lc_infer_sampling(lc_lk: lk.LightCurve):
    '''Infer sampling period from light curve'''
    time_diffs = lc_lk['time'][1:] - lc_lk['time'][:-1]
    return np.median(time_diffs)

## Data processing

### GTO data loading

In [6]:
# Load GTO data table:
gto = pd.read_csv(GTO_FILE, sep=',', decimal='.')
gto.head(5)

,Karmn,Name,Comp,GJ,RA_J2016_deg,DE_J2016_deg,RA_J2000,DE_J2000,l_J2016_deg,b_J2016_deg,...,T0_PG_TESS,e_T0_PG_TESS,offset_PG_TESS,e_offset_PG_TESS,FAP_PG_TESS,valid_PG_TESS,error_PG_TESS,elapsed_time_PG_TESS,fits_file_TESS,fig_file_TESS
0,J23585+076,Wolf 1051,AB,4383,359.636420,7.656947,23:58:32.65,+07:39:30.1,100.839686,-52.931049,...,NaN,NaN,NaN,NaN,NaN,0.0,Not recognized as a supported data product:\nn...,0.002992,NaN,NaN
1,J23556-061,GJ 912,AB,912,358.913617,-6.144283,23:55:39.78,-06:08:33.4,88.129933,-65.175491,...,NaN,NaN,NaN,NaN,NaN,0.0,Not recognized as a supported data product:\nn...,0.001671,NaN,NaN
2,J23548+385,RX J2354.8+3831,-,NaN,358.713658,38.526340,23:54:51.46,+38:31:36.2,110.941908,-23.024449,...,1764.609498,0.010704,8262.219751,1.365667,1.000000,1.0,NaN,344.002685,../data/CARMENES_GTO_TESS_PGs/J23548+385_TESS_...,../data/CARMENES_GTO_TESS_PGs/figures/J23548+3...
3,J23505-095,LP 763-012,-,4367,357.634705,-9.560964,23:50:31.64,-09:33:32.7,80.777067,-67.303426,...,1354.108815,0.001261,7767.134654,0.094298,0.064148,1.0,NaN,473.533042,../data/CARMENES_GTO_TESS_PGs/J23505-095_TESS_...,../data/CARMENES_GTO_TESS_PGs/figures/J23505-0...
4,J23492+024,BR Psc,-,908,357.306604,2.396918,23:49:12.53,+02:24:04.4,93.567467,-56.885396,...,NaN,NaN,NaN,NaN,NaN,0.0,Not recognized as a supported data product:\nn...,0.000997,NaN,NaN


In [7]:
print(list(gto.columns))

['Karmn', 'Name', 'Comp', 'GJ', 'RA_J2016_deg', 'DE_J2016_deg', 'RA_J2000', 'DE_J2000', 'l_J2016_deg', 'b_J2016_deg', 'Ref01', 'SpT', 'SpTnum', 'Ref02', 'Teff_K', 'eTeff_K', 'logg', 'elogg', '[Fe/H]', 'e[Fe/H]', 'Ref03', 'L_Lsol', 'eL_Lsol', 'Ref04', 'R_Rsol', 'eR_Rsol', 'Ref05', 'M_Msol', 'eM_Msol', 'Ref06', 'muRA_masa-1', 'emuRA_masa-1', 'muDE_masa-1', 'emuDE_masa-1', 'Ref07', 'pi_mas', 'epi_mas', 'Ref08', 'd_pc', 'ed_pc', 'Ref09', 'Vr_kms-1', 'eVr_kms-1', 'Ref10', 'ruwe', 'Ref11', 'U_kms-1', 'eU_kms-1', 'V_kms-1', 'eV_kms-1', 'W_kms-1', 'eW_kms-1', 'Ref12', 'sa_m/s/a', 'esa_m/s/a', 'Ref13', 'SKG', 'Ref14', 'SKG_lit', 'Ref14_lit', 'Pop', 'Ref15', 'vsini_flag', 'vsini_kms-1', 'evsini_kms-1', 'Ref16', 'P_d', 'eP_d', 'Ref17', 'pEWHalpha_A', 'epEWHalpha_A', 'Ref18', 'log(LHalpha/Lbol)', 'elog(LHalpha/Lbol)', 'Ref19', '1RXS', 'CRT_s-1', 'eCRT_s-1', 'HR1', 'eHR1', 'HR2', 'eHR2', 'Flux_X_E-13_ergcm-2s-1', 'eFlux_X_E-13_ergcm-2s-1', 'LX/LJ', 'eLX/LJ', 'Ref20', 'Activity', 'Ref21', 'FUV_mag',

In [8]:
gto.loc[72, ['Karmn', 'has_lc', 'lc_file']]

Karmn                                             J19025+754
has_lc                                                  True
lc_file    ../data/CARMENES_GTO_TESS_lc/lightcurves/J1902...
Name: 72, dtype: object

Generate the proper auxiliary columns (with the basic periodograms results).

In [9]:
# Additional columns:
if 'n_TESS' in gto.columns:
    pass
else:
    gto['n_TESS'] = None # Number of points in TESS LC
    gto['Ps_TESS'] = None # Sampling period (d)
    gto['fs_TESS'] = None # Sampling frequency (d^(-1))
    gto['wmean_TESS'] = None # Mean of TESS LC
    gto['wrms_TESS'] = None # RMS of TESS LC
    gto['info_PG_TESS'] = None # Information text about the PG
    gto['maxP_PG_TESS'] = None # Max power value in the PG
    gto['maxSNR_PG_TESS'] = None # Max power value in the PG
    gto['rms_PG_TESS'] = None # RMS value in the PG residuals
    gto['f_PG_TESS'] = None # Best frequency in the PG (d^(-1))
    gto['e_f_PG_TESS'] = None # Error of the best frequency in the PG (d^(-1))
    gto['Pd_PG_TESS'] = None # Best period in the PG (d)
    gto['e_Pd_PG_TESS'] = None # Error of the best period in the PG (d)
    gto['Ph_PG_TESS'] = None # Best period in the PG (hours)
    gto['e_Ph_PG_TESS'] = None # Error of the best period in the PG (hours)
    gto['Pm_PG_TESS'] = None # Best period in the PG (minutes)
    gto['e_Pm_PG_TESS'] = None # Error of the best period in the PG (minutes)
    gto['A_PG_TESS'] = None # Amplitude of the best frequency
    gto['e_A_PG_TESS'] = None # Error of the amplitude of the best frequency
    gto['ph_PG_TESS'] = None # Amplitude of the best frequency
    gto['e_ph_PG_TESS'] = None # Error of the amplitude of the best frequency
    gto['T0_PG_TESS'] = None # Reference epoch of the best frequency
    gto['e_T0_PG_TESS'] = None # Error of the epoch of the best frequency
    gto['offset_PG_TESS'] = None # Offset of the best frequency
    gto['e_offset_PG_TESS'] = None # Error of the offset of the best frequency
    gto['FAP_PG_TESS'] = None # False alarm probability
    gto['valid_PG_TESS'] = None # Flag to indicate if the periodogram calculation succeeded (1) or not (0).
    gto['error_PG_TESS'] = None # The error raised during processing. Empty if processing was successful.
    gto['elapsed_time_PG_TESS'] = None # The time elapsed in calculation
    gto['fits_file_TESS'] = None # The name of the processed file.
    gto['fig_file_TESS'] = None # The name of the figure file.


In [10]:
print(list(gto.columns))

['Karmn', 'Name', 'Comp', 'GJ', 'RA_J2016_deg', 'DE_J2016_deg', 'RA_J2000', 'DE_J2000', 'l_J2016_deg', 'b_J2016_deg', 'Ref01', 'SpT', 'SpTnum', 'Ref02', 'Teff_K', 'eTeff_K', 'logg', 'elogg', '[Fe/H]', 'e[Fe/H]', 'Ref03', 'L_Lsol', 'eL_Lsol', 'Ref04', 'R_Rsol', 'eR_Rsol', 'Ref05', 'M_Msol', 'eM_Msol', 'Ref06', 'muRA_masa-1', 'emuRA_masa-1', 'muDE_masa-1', 'emuDE_masa-1', 'Ref07', 'pi_mas', 'epi_mas', 'Ref08', 'd_pc', 'ed_pc', 'Ref09', 'Vr_kms-1', 'eVr_kms-1', 'Ref10', 'ruwe', 'Ref11', 'U_kms-1', 'eU_kms-1', 'V_kms-1', 'eV_kms-1', 'W_kms-1', 'eW_kms-1', 'Ref12', 'sa_m/s/a', 'esa_m/s/a', 'Ref13', 'SKG', 'Ref14', 'SKG_lit', 'Ref14_lit', 'Pop', 'Ref15', 'vsini_flag', 'vsini_kms-1', 'evsini_kms-1', 'Ref16', 'P_d', 'eP_d', 'Ref17', 'pEWHalpha_A', 'epEWHalpha_A', 'Ref18', 'log(LHalpha/Lbol)', 'elog(LHalpha/Lbol)', 'Ref19', '1RXS', 'CRT_s-1', 'eCRT_s-1', 'HR1', 'eHR1', 'HR2', 'eHR2', 'Flux_X_E-13_ergcm-2s-1', 'eFlux_X_E-13_ergcm-2s-1', 'LX/LJ', 'eLX/LJ', 'Ref20', 'Activity', 'Ref21', 'FUV_mag',

### Load previous reference GTO objetcs data

In [11]:
ref_gto = pd.read_csv(REF_GTO_FILE, sep=',', decimal='.')
ref_gto.head(5)

,Karmn,Name,Comp,GJ,RA_J2016_deg,DE_J2016_deg,RA_J2000,DE_J2000,l_J2016_deg,b_J2016_deg,...,T0_PG_TESS,e_T0_PG_TESS,offset_PG_TESS,e_offset_PG_TESS,FAP_PG_TESS,valid_PG_TESS,error_PG_TESS,elapsed_time_PG_TESS,fits_file_TESS,fig_file_TESS
0,J23585+076,Wolf 1051,AB,4383,359.636420,7.656947,23:58:32.65,+07:39:30.1,100.839686,-52.931049,...,NaN,NaN,NaN,NaN,NaN,0.0,Not recognized as a supported data product:\nn...,0.002199,NaN,NaN
1,J23556-061,GJ 912,AB,912,358.913617,-6.144283,23:55:39.78,-06:08:33.4,88.129933,-65.175491,...,NaN,NaN,NaN,NaN,NaN,0.0,Not recognized as a supported data product:\nn...,0.000974,NaN,NaN
2,J23548+385,RX J2354.8+3831,-,NaN,358.713658,38.526340,23:54:51.46,+38:31:36.2,110.941908,-23.024449,...,1764.609498,0.010704,8262.219751,1.365667,1.000000,1.0,NaN,344.002685,../data/CARMENES_GTO_TESS_PGs/J23548+385_TESS_...,../data/CARMENES_GTO_TESS_PGs/figures/J23548+3...
3,J23505-095,LP 763-012,-,4367,357.634705,-9.560964,23:50:31.64,-09:33:32.7,80.777067,-67.303426,...,1354.108815,0.001261,7767.134654,0.094298,0.064148,1.0,NaN,473.533042,../data/CARMENES_GTO_TESS_PGs/J23505-095_TESS_...,../data/CARMENES_GTO_TESS_PGs/figures/J23505-0...
4,J23492+024,BR Psc,-,908,357.306604,2.396918,23:49:12.53,+02:24:04.4,93.567467,-56.885396,...,NaN,NaN,NaN,NaN,NaN,0.0,Not recognized as a supported data product:\nn...,0.000984,NaN,NaN


### Batch processing of all TESS files

In [12]:
n = len(gto)
n

390

#### Batch processing

In [13]:
# Batch processing:
lapse_list = []
median_lapse = None
previous_objects = ref_gto['Karmn'].to_list()
tess_columns = [c for c in list(gto.columns) if '_TESS' in c]
#for i in range(0, 3): # TEST
#for i in range(0, 72): # Record 72 took too long to complete.
#for i in range(73, 176): # The final records were taking much longer to complete.
#for i in range(176, 246): # Aborted at 246 for convenience
#for i in range(246, 250): # Record 250 took too long to complete.
#for i in range(251, 300): # Record 250 took too long to complete.
#for i in range(300, len(gto)):
for i in range(0, len(gto)):
#for i in range(385, 386): # TEST
    start_time = time.time()
    # Names:
    karmn = gto.loc[i, 'Karmn'] # KARMENES name
    commn = gto.loc[i, 'Name'] # Common name
    tic_id = str(gto.loc[i, 'TIC_id']) # TESS TIC identifier
    # Check if this record already has correct values:
    if gto.loc[i, 'valid_PG_TESS'] == 1:
        # Skip the already calculated record:
        continue # Go to the next element in the loop
    # Check if the TESS curves have been calculated previously:
    if karmn in previous_objects and ref_gto[ref_gto['Karmn']==karmn]['valid_PG_TESS'].values == 1:
        # Just copy the values (periodogram files and figures should already be in the relevant folders):
        #print("Record: %d already calculated in previous objects, copying values..." %i)

        for col in tess_columns:
            gto.loc[i, col] = ref_gto.loc[ref_gto['Karmn']==karmn, col].values
        #gto.loc[i, tess_columns] = ref_gto.loc[ref_gto['Karmn']==karmn, tess_columns].values

        continue # Go to the next element in the loop

    print("Record: %d, started at %s"
          %(i, time.strftime('%d/%m/%Y, %H:%M:%S', time.localtime(start_time))))
    if median_lapse is None:
        print("Previous median lapse time: %s" %median_lapse)
    else:
        print("Previous median lapse time: %.2f seconds" %median_lapse)
    print("Processing %s star (TIC: %s)..." %(karmn, tic_id))
    try:
        # LOAD LC FILE:
        lc_file = gto.loc[i, 'lc_file']
        print("filename: %s" %lc_file)
        lc_lk = tess_lc_load(lc_file)
        
        # GENERATE PERIODOGRAM:
        gls = Gls((lc_lk['time'].value, lc_lk['flux'].value, lc_lk['flux_err'].value),
              fbeg=FBEG, fend=FEND, Pbeg=PBEG, Pend=PEND, ofac=OFAC, hifac=HIFAC, freq=FREQ,
              norm=NORM, ls=LS, fast=FAST, verbose=VERBOSE)
        
        # GENERATE THE FITS FILE:
        # Prepare the fits primary HDU (only header):
        primary_header = fits.Header()
        primary_header['OBJECT'] = (karmn, "KARMENES target name")
        primary_header['NAME'] = (commn, "Object common name")
        primary_header['TIC'] = (tic_id, "Object TESS identifier")
        primary_header['RA_J2000'] = ("00:05:10.89", "Object right ascension (J2000)")
        primary_header['DE_J2000'] = ("+45:47:11.6", "Object declination (J2000)")
        primary_header['SPTYPE'] = ("M1.0 V", "Spectral type")
        primary_header['TEFF_K'] = (3773, "Effective temperature in Kelvin")
        primary_header['LOGG'] = (5.07, "Logarithm of surface gravity")
        primary_header['FEH'] = (-0.04, "Metallicity")
        primary_header['L_LSUN'] = (0.0436229, "Luminosity in Solar luminosities")
        primary_header['R_RSUN'] = (0.48881, "Radius in Solar radii")
        primary_header['M_MSUN'] = (0.4918, "Mass in Solar masses")
        primary_header['D_PC'] = (11.50352803, "Distance in parsec")
        hdu_primary = fits.PrimaryHDU(header=primary_header)

        # Prepare the LC HDU:
        hdu_lc = fits.table_to_hdu(QTable(lc_lk.to_table()))
        hdu_lc.name = "TESS_LC"
        psample = lc_infer_sampling(lc_lk).value
        fsample = 1.0 / psample
        fnyq = 2.0 * fsample
        freq_units = u.d ** (-1)
        hdu_lc.header['OBJECT'] = (karmn, "KARMENES target name")
        hdu_lc.header['PUNIT'] = u.d.to_string(format='fits')
        hdu_lc.header['FUNIT'] = freq_units.to_string(format='fits')
        hdu_lc.header['RVPOINTS'] = (gls.N, "Number of points in the RV curve")
        hdu_lc.header['AVGFLUX'] = (gls._Y, "Average flux of RV curve")
        hdu_lc.header['RMSFLUX'] = (gls._YY, "Flux RMS of RV curve")
        hdu_lc.header['PSAMPLE'] = (psample, "Inferred cadence in RV curve")
        hdu_lc.header['FSAMPLE'] = (fsample, "Inferred sampling frequency in RV curve")
        hdu_lc.header['FNYQUIST'] = (fnyq, "Calculated Nyquist frequency value")
                
        # Prepare the PG HDU:
        hdu_pg = fits.table_to_hdu(
            QTable(data=[gls.freq, gls.power], names=['freq', 'power'], 
                   units=[1.0 / u.d, (u.electron / u.s) ** 2]))
        hdu_pg.name = "GLS_PG"
        fpoints = len(gls.f)
        fres = (gls.fend - gls.fbeg) / (fpoints - 1)
        hdu_pg.header['OBJECT'] = (karmn, "KARMENES target name")
        hdu_pg.header['FUNIT'] = (freq_units.to_string(format='fits'), "Unit for frequencies")
        hdu_pg.header['PUNIT'] = (u.d.to_string(format='fits'), "Unit for periods")
        hdu_pg.header['PK_FREQ'] = (gls.best['f'], "Frequency of the peak in periodogram")
        hdu_pg.header['PK_POW'] = (gls.pmax, "Power of the peak in periodogram")
        hdu_pg.header['PK_SNR'] = (gls.best['amp'] / gls.rms, "SNR of the peak in periodogram")
        hdu_pg.header['PK_FAP'] = (gls.FAP(Pn=None), "FAP of the peak in periodogram")
        hdu_pg.header['RES_RMS'] = (gls.rms, "RMS of residuals in periodogram")
        hdu_pg.header['FSAMPLE'] = (fsample, "Inferred sampling frequency in TESS curve")
        hdu_pg.header['FNYQUIST'] = (fnyq, "Calculated Nyquist frequency value")
        hdu_pg.header['FPOINTS'] = (fpoints, "Number of points in periodogram")
        hdu_pg.header['FBEG'] = (gls.fbeg, "Start frequency in periodogram")
        hdu_pg.header['FEND'] = (gls.fend, "End frequency in periodogram")
        hdu_pg.header['FRES'] = (fres, "Frequency resolution in periodogram")
        hdu_pg.header['F'] = (gls.best['f'], "Peak best estimate: frequency")
        hdu_pg.header['E_F'] = (gls.best['e_f'], "Peak best estimate: frequency error")
        hdu_pg.header['P'] = (gls.best['P'], "Peak best estimate: period")
        hdu_pg.header['E_P'] = (gls.best['e_P'], "Peak best estimate: period error")
        hdu_pg.header['A'] = (gls.best['amp'], "Peak best estimate: amplitude")
        hdu_pg.header['E_A'] = (gls.best['e_amp'], "Peak best estimate: amplitude error")
        hdu_pg.header['PH'] = (gls.best['ph'], "Peak best estimate: phase")
        hdu_pg.header['E_PH'] = (gls.best['e_ph'], "Peak best estimate: phase error")
        hdu_pg.header['T0'] = (gls.best['T0'], "Peak best estimate: frequency")
        hdu_pg.header['E_T0'] = (gls.best['e_T0'], "Peak best estimate: frequency error")
        hdu_pg.header['OFF'] = (gls.best['offset'], "Peak best estimate: offset")
        hdu_pg.header['E_OFF'] = (gls.best['e_offset'], "Peak best estimate: offset error")
        hdu_pg.header['OFAC'] = (gls.ofac, "Setup: oversampling factor")
        hdu_pg.header['HIFAC'] = (gls.ofac, "Setup: maximum frequency factor")
        hdu_pg.header['NORM'] = (gls.norm, "Setup: normalization type")
        hdu_pg.header['LS'] = (gls.ls, "Setup: conventional Lomb-Scargle calculation")
        hdu_pg.header['FAST'] = (gls.fast, "Setup: fast evaluation, recursive trigonometric")
        
        # Create and save the fits file:
        fits_filename = OUT_PROCESSED_FOLDER + karmn + "_TESS_PG.fits"
        hdul = fits.HDUList([hdu_primary, hdu_lc, hdu_pg])
        hdul.writeto(fits_filename, overwrite=True)
        
        # Delete all the HDUs:
        del hdul
        del hdu_primary
        del hdu_lc
        del hdu_pg

        # FILL IN THE DATA IN THE GTO TABLE:
        gto.loc[i, 'n_TESS'] = gls.N
        gto.loc[i, 'Ps_TESS'] = psample
        gto.loc[i, 'fs_TESS'] = fsample
        gto.loc[i, 'wmean_TESS'] = gls._Y
        gto.loc[i, 'wrms_TESS'] = gls._YY
        gto.loc[i, 'info_PG_TESS'] = gls.info(stdout=False)
        gto.loc[i, 'maxP_PG_TESS'] = gls.power.max()
        gto.loc[i, 'maxSNR_PG_TESS'] = gls.best['amp'] / gls.rms
        gto.loc[i, 'rms_PG_TESS'] = gls.rms
        gto.loc[i, 'f_PG_TESS'] = gls.best['f']
        gto.loc[i, 'e_f_PG_TESS'] = gls.best['e_f']
        gto.loc[i, 'Pd_PG_TESS'] = gls.best['P']
        gto.loc[i, 'e_Pd_PG_TESS'] = gls.best['e_P']
        gto.loc[i, 'Ph_PG_TESS'] = 24.0 * gls.best['P']
        gto.loc[i, 'e_Ph_PG_TESS'] = 24.0 * gls.best['e_P']
        gto.loc[i, 'Pm_PG_TESS'] = 24.0 * 60.0 * gls.best['P']
        gto.loc[i, 'e_Pm_PG_TESS'] = 24.0 * 60.0 * gls.best['e_P']
        gto.loc[i, 'A_PG_TESS'] = gls.best['amp']
        gto.loc[i, 'e_A_PG_TESS'] = gls.best['e_amp']
        gto.loc[i, 'ph_PG_TESS'] = gls.best['ph']
        gto.loc[i, 'e_ph_PG_TESS'] = gls.best['e_ph']
        gto.loc[i, 'T0_PG_TESS'] = gls.best['T0']
        gto.loc[i, 'e_T0_PG_TESS'] = gls.best['e_T0']
        gto.loc[i, 'offset_PG_TESS'] = gls.best['offset']
        gto.loc[i, 'e_offset_PG_TESS'] = gls.best['e_offset']
        gto.loc[i, 'FAP_PG_TESS'] = gls.FAP(Pn=None)

        # GENERATE THE FIGURE:
        fig = gls.plot(block=False, period=False,
                       fap=FAP_LEVELS_PLOT, gls=True, data=True, residuals=True)
        figure_title = "TESS LC: %s (TIC %s), P=%.4f [min], FAP=%.4f%%" \
            %(karmn, tic_id, gto.loc[i, 'Pm_PG_TESS'], 100.0 * gto.loc[i, 'FAP_PG_TESS'])
        fig.suptitle(figure_title, fontdict = {'fontsize' : 36})
        fig.tight_layout()
        # Save the figure to disk:
        fig_file = OUT_IMG_FOLDER + karmn + "_TESS_PG.png"
        fig.savefig(fig_file)
        plt.close() # Prevent the figure from showing.

        # SET THE RECORD CALCULATION AS VALID AND STORE THE REULTING FILENAMES
        gto.loc[i, 'valid_PG_TESS'] = 1
        gto.loc[i, 'error_PG_TESS'] = ""
        gto.loc[i, 'fits_file_TESS'] = fits_filename
        gto.loc[i, 'fig_file_TESS'] = fig_file

        # UPDATE THE AVERAGE RECORD PROCESSING TIME:
        lapse = time.time() - start_time
        lapse_list.append(lapse)
        median_lapse = np.nanmedian(lapse_list)
        gto.loc[i, 'elapsed_time_PG_TESS'] = lapse
        
        # SAVE THE UPDATED GTO TABLE TO DISK:
        gto.to_csv(GTO_FILE, sep=',', decimal='.', index=False)
        
        # Report successful execution:
        print("Elapsed time: %.2f seconds" %lapse)
        print("... SUCCESS.")
        
    except Exception as e:
        # Some error happened, establish the record as not valid and record the error:
        error = "--- Some ERROR happened with record #%d, %s star. Error=%s" %(i, karmn, e)
        print(error)
        gto.loc[i, 'valid_PG_TESS'] = 0
        gto.loc[i, 'error_PG_TESS'] = e
        
        # Try to update the record, and save the file:
        try:
            # UPDATE THE AVERAGE RECORD PROCESSING TIME:
            lapse = time.time() - start_time
            lapse_list.append(lapse)
            median_lapse = np.nanmedian(lapse_list)
            gto.loc[i, 'elapsed_time_PG_TESS'] = lapse
            print("Elapsed time: %.2f seconds" %lapse)

            # SAVE THE UPDATED GTO TABLE TO DISK:
            gto.to_csv(GTO_FILE, sep=',', decimal='.', index=False)
        except Exception as e2:
            error_2 = "--- Additional ERROR happened with record #%d, %s star. Error=%s" %(i, karmn, str(e2))
            print(error)
            gto.loc[i, 'error_PG_TESS'] = gto.loc[i, 'error_PG_TESS'] + "/" + str(e2)



Record: 0, started at 17/06/2022, 09:11:39
Previous median lapse time: None
Processing J23585+076 star (TIC: 408486498)...
filename: nan
--- Some ERROR happened with record #0, J23585+076 star. Error=Not recognized as a supported data product:
nan
This file may be corrupt due to an interrupted download. Please remove it from your disk and try again.
Elapsed time: 0.00 seconds
Record: 1, started at 17/06/2022, 09:11:40
Previous median lapse time: 0.00 seconds
Processing J23556-061 star (TIC: 9759680)...
filename: nan
--- Some ERROR happened with record #1, J23556-061 star. Error=Not recognized as a supported data product:
nan
This file may be corrupt due to an interrupted download. Please remove it from your disk and try again.
Elapsed time: 0.00 seconds
Record: 4, started at 17/06/2022, 09:11:40
Previous median lapse time: 0.00 seconds
Processing J23492+024 star (TIC: 267255253)...
filename: nan
--- Some ERROR happened with record #4, J23492+024 star. Error=Not recognized as a supporte

Record: 45, started at 17/06/2022, 09:11:41
Previous median lapse time: 0.00 seconds
Processing J20556-140N star (TIC: 1995862013)...
filename: nan
--- Some ERROR happened with record #45, J20556-140N star. Error=Not recognized as a supported data product:
nan
This file may be corrupt due to an interrupted download. Please remove it from your disk and try again.
Elapsed time: 0.00 seconds
Record: 47, started at 17/06/2022, 09:11:41
Previous median lapse time: 0.00 seconds
Processing J20525-169 star (TIC: 262016943)...
filename: nan
--- Some ERROR happened with record #47, J20525-169 star. Error=Not recognized as a supported data product:
nan
This file may be corrupt due to an interrupted download. Please remove it from your disk and try again.
Elapsed time: 0.00 seconds
Record: 49, started at 17/06/2022, 09:11:41
Previous median lapse time: 0.00 seconds
Processing J20450+444 star (TIC: 239332587)...
filename: ../data/CARMENES_GTO_TESS_lc/lightcurves/J20450+444/J20450+444_TESS_TIC239332

c:\users\ciroe\gls.py:368: RuntimeWarning: invalid value encountered in sqrt
  self.rms = rms = sqrt(self._YY*(1.-pmax))
c:\users\ciroe\gls.py:394: RuntimeWarning: invalid value encountered in sqrt
  p["e_f"] = e_f = sqrt(-2./self.N / aa * (1.-self.pmax))


--- Some ERROR happened with record #49, J20450+444 star. Error=Floating point nan values are not allowed in FITS headers.
Elapsed time: 28.10 seconds
Record: 50, started at 17/06/2022, 09:12:09
Previous median lapse time: 0.00 seconds
Processing J20405+154 star (TIC: 340807280)...
filename: nan
--- Some ERROR happened with record #50, J20405+154 star. Error=Not recognized as a supported data product:
nan
This file may be corrupt due to an interrupted download. Please remove it from your disk and try again.
Elapsed time: 0.00 seconds
Record: 56, started at 17/06/2022, 09:12:09
Previous median lapse time: 0.00 seconds
Processing J20093-012 star (TIC: 243800364)...
filename: nan
--- Some ERROR happened with record #56, J20093-012 star. Error=Not recognized as a supported data product:
nan
This file may be corrupt due to an interrupted download. Please remove it from your disk and try again.
Elapsed time: 0.00 seconds
Record: 57, started at 17/06/2022, 09:12:09
Previous median lapse time:

c:\users\ciroe\gls.py:368: RuntimeWarning: invalid value encountered in sqrt
  self.rms = rms = sqrt(self._YY*(1.-pmax))
c:\users\ciroe\gls.py:394: RuntimeWarning: invalid value encountered in sqrt
  p["e_f"] = e_f = sqrt(-2./self.N / aa * (1.-self.pmax))


...

Elapsed time: 444.88 seconds
... SUCCESS.
Record: 276, started at 17/06/2022, 10:06:56
Previous median lapse time: 0.00 seconds
Processing J07386-212 star (TIC: 415085843)...
filename: nan
--- Some ERROR happened with record #276, J07386-212 star. Error=Not recognized as a supported data product:
nan
This file may be corrupt due to an interrupted download. Please remove it from your disk and try again.
Elapsed time: 0.00 seconds
Record: 278, started at 17/06/2022, 10:06:57
Previous median lapse time: 0.00 seconds
Processing J07353+548 star (TIC: 355061271)...
filename: nan
--- Some ERROR happened with record #278, J07353+548 star. Error=Not recognized as a supported data product:
nan
This file may be corrupt due to an interrupted download. Please remove it from your disk and try again.
Elapsed time: 0.00 seconds
Record: 286, started at 17/06/2022, 10:06:57
Previous median lapse time: 0.00 seconds
Processing J06594+193 star (TIC: 58998044)...
filename: nan
--- Some ERROR happened with r

c:\users\ciroe\gls.py:642: UserWarning: Matplotlib is currently using module://ipykernel.pylab.backend_inline, which is a non-GUI backend, so cannot show the figure.
  fig.show()


Elapsed time: 407.50 seconds
... SUCCESS.
Record: 351, started at 17/06/2022, 10:13:45
Previous median lapse time: 0.00 seconds
Processing J02489-145E star (TIC: 29960109)...
filename: ../data/CARMENES_GTO_TESS_lc/lightcurves/J02489-145E/J02489-145E_TESS_TIC29960109_S04.fits


c:\users\ciroe\gls.py:642: UserWarning: Matplotlib is currently using module://ipykernel.pylab.backend_inline, which is a non-GUI backend, so cannot show the figure.
  fig.show()


Elapsed time: 429.79 seconds
... SUCCESS.
Record: 353, started at 17/06/2022, 10:20:55
Previous median lapse time: 0.00 seconds
Processing J02465+164 star (TIC: 218088471)...
filename: nan
--- Some ERROR happened with record #353, J02465+164 star. Error=Not recognized as a supported data product:
nan
This file may be corrupt due to an interrupted download. Please remove it from your disk and try again.
Elapsed time: 0.00 seconds
Record: 356, started at 17/06/2022, 10:20:55
Previous median lapse time: 0.00 seconds
Processing J02358+202 star (TIC: 247042708)...
filename: nan
--- Some ERROR happened with record #356, J02358+202 star. Error=Not recognized as a supported data product:
nan
This file may be corrupt due to an interrupted download. Please remove it from your disk and try again.
Elapsed time: 0.00 seconds
Record: 359, started at 17/06/2022, 10:20:55
Previous median lapse time: 0.00 seconds
Processing J02164+135 star (TIC: 408587159)...
filename: nan
--- Some ERROR happened with 

## Review a given file

In [14]:
object_name = "J23548+385"

In [15]:
read_hdul = fits.open(OUT_PROCESSED_FOLDER + object_name + "_TESS_PG.fits",
                      mode='update', memmap=False)
read_hdul

[<astropy.io.fits.hdu.image.PrimaryHDU object at 0x000001755C4FB940>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x000001755C474250>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x0000017556396640>]

In [16]:
read_hdul.info()

Filename: ../data/CARMENES_GTO_TESS_PGs/J23548+385_TESS_PG.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      17   ()      
  1  TESS_LC       1 BinTableHDU    217   12878R x 24C   [2D, E, E, E, J, D, D, E, E, E, E, E, E, J, D, E, D, E, D, E, D, E, E, E]   
  2  GLS_PG        1 BinTableHDU     45   1000001R x 2C   [D, D]   


### Retrieve basic information

In [17]:
read_hdul['PRIMARY'].header

SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                    8 / array data type                                
NAXIS   =                    0 / number of array dimensions                     
EXTEND  =                    T                                                  
OBJECT  = 'J23548+385'         / KARMENES target name                           
NAME    = 'RX J2354.8+3831'    / Object common name                             
TIC     = '291689873'          / Object TESS identifier                         
RA_J2000= '00:05:10.89'        / Object right ascension (J2000)                 
DE_J2000= '+45:47:11.6'        / Object declination (J2000)                     
SPTYPE  = 'M1.0 V  '           / Spectral type                                  
TEFF_K  =                 3773 / Effective temperature in Kelvin                
LOGG    =                 5.07 / Logarithm of surface gravity                   
FEH     =                -0.

### Retrieve time series curve information

In [18]:
read_hdul['TESS_LC']

In [19]:
read_hdul['TESS_LC'].header

XTENSION= 'BINTABLE'           / binary table extension                         
BITPIX  =                    8 / array data type                                
NAXIS   =                    2 / number of array dimensions                     
NAXIS1  =                  132 / length of dimension 1                          
NAXIS2  =                12878 / length of dimension 2                          
PCOUNT  =                    0 / number of group parameters                     
GCOUNT  =                    1 / number of groups                               
TFIELDS =                   24 / number of table fields                         
TIMESYS = 'TDB     '           / Default time scale                             
JDREF   =                  0.0 / Time columns are jd = jd1 + jd2                
TREFPOS = 'TOPOCENTER'         / Time reference position                        
TTYPE1  = 'time    '                                                            
TFORM1  = '2D      '        

In [20]:
read_hdul['TESS_LC'].data[:10]

FITS_rec([([ 2.45876500e+06, -3.10855424e-01], 8036.715 , 14.602657, 0.0047049 , 386805, 743.37405765, 1030.75688962, 7419.3154, 13.216773, 1896.0168, 6.645701 , 8036.715 , 14.602657, 0, nan, nan, nan, nan, 743.37405765, 0.00165376, 1030.75688962, 0.00156959, 0.04306674, 0.08566172),
          ([ 2.45876500e+06, -3.09466527e-01], 8045.7295, 14.630568, 0.00470491, 386806, 743.37714375, 1030.75655315, 7433.0312, 13.242034, 1900.8579, 6.6685295, 8045.7295, 14.630568, 0, nan, nan, nan, nan, 743.37714375, 0.00165215, 1030.75655315, 0.0015669 , 0.04319848, 0.08758464),
          ([ 2.45876500e+06, -3.08077632e-01], 8045.2686, 14.622687, 0.00470492, 386807, 743.38027732, 1030.75631546, 7430.282 , 13.234902, 1898.9921, 6.6640596, 8045.2686, 14.622687, 0, nan, nan, nan, nan, 743.38027732, 0.00165761, 1030.75631546, 0.00156569, 0.04801591, 0.08800385),
          ([ 2.45876500e+06, -3.06688735e-01], 8035.3765, 14.61121 , 0.00470493, 386808, 743.37142282, 1030.75155629, 7416.531 , 13.224514, 1900.

In [21]:
read_rv_table = QTable.read(read_hdul['TESS_LC'])
read_rv_table

time [2],flux,flux_err,timecorr,cadenceno,centroid_col,centroid_row,sap_flux,sap_flux_err,sap_bkg,sap_bkg_err,pdcsap_flux,pdcsap_flux_err,quality,psf_centr1,psf_centr1_err,psf_centr2,psf_centr2_err,mom_centr1,mom_centr1_err,mom_centr2,mom_centr2_err,pos_corr1,pos_corr2
d,,,d,,pix,pix,,,,,,,,pix,pix,pix,pix,pix,pix,pix,pix,pix,pix
float64,float32,float32,float32,int32,float64,float64,float32,float32,float32,float32,float32,float32,int32,float64,float32,float64,float32,float64,float32,float64,float32,float32,float32
2458765.0 .. -0.3108554236009695,8.0367148e+03,1.4602657e+01,4.7049029e-03,386805,743.37406,1030.75689,7.4193154e+03,1.3216773e+01,1.8960168e+03,6.6457009e+00,8.0367148e+03,1.4602657e+01,0,nan,nan,nan,nan,743.37406,1.6537585e-03,1030.75689,1.5695895e-03,4.3066736e-02,8.5661724e-02
2458765.0 .. -0.30946652739862657,8.0457295e+03,1.4630568e+01,4.7049108e-03,386806,743.37714,1030.75655,7.4330312e+03,1.3242034e+01,1.9008579e+03,6.6685295e+00,8.0457295e+03,1.4630568e+01,0,nan,nan,nan,nan,743.37714,1.6521512e-03,1030.75655,1.5668995e-03,4.3198477e-02,8.7584637e-02
2458765.0 .. -0.3080776316621723,8.0452686e+03,1.4622687e+01,4.7049182e-03,386807,743.38028,1030.75632,7.4302822e+03,1.3234902e+01,1.8989921e+03,6.6640596e+00,8.0452686e+03,1.4622687e+01,0,nan,nan,nan,nan,743.38028,1.6576147e-03,1030.75632,1.5656911e-03,4.8015915e-02,8.8003851e-02
2458765.0 .. -0.3066887354593746,8.0353765e+03,1.4611210e+01,4.7049262e-03,386808,743.37142,1030.75156,7.4165308e+03,1.3224514e+01,1.9003071e+03,6.6757946e+00,8.0353765e+03,1.4611210e+01,0,nan,nan,nan,nan,743.37142,1.6582146e-03,1030.75156,1.5672606e-03,3.9467834e-02,8.1147596e-02
2458765.0 .. -0.3052998392563495,8.0455757e+03,1.4600376e+01,4.7049341e-03,386809,743.37579,1030.75775,7.4285151e+03,1.3214708e+01,1.8975846e+03,6.6496892e+00,8.0455757e+03,1.4600376e+01,0,nan,nan,nan,nan,743.37579,1.6511347e-03,1030.75775,1.5659113e-03,4.2884134e-02,8.6799361e-02
2458765.0 .. -0.3039109435198952,8.0578491e+03,1.4611950e+01,4.7049415e-03,386810,743.38068,1030.75563,7.4400063e+03,1.3225183e+01,1.8851906e+03,6.6451707e+00,8.0578491e+03,1.4611950e+01,0,nan,nan,nan,nan,743.38068,1.6487412e-03,1030.75563,1.5666722e-03,4.6368241e-02,8.7138519e-02
2458765.0 .. -0.30252204731777965,8.0199448e+03,1.4623798e+01,4.7049494e-03,386811,743.38077,1030.75486,7.4108506e+03,1.3235908e+01,1.8932153e+03,6.6664557e+00,8.0199448e+03,1.4623798e+01,0,nan,nan,nan,nan,743.38077,1.6638068e-03,1030.75486,1.5711738e-03,4.3958057e-02,8.7265849e-02
2458765.0 .. -0.3011331511154367,8.0214658e+03,1.4593859e+01,4.7049574e-03,386812,743.37518,1030.75434,7.4129116e+03,1.3208809e+01,1.8983730e+03,6.6483836e+00,8.0214658e+03,1.4593859e+01,0,nan,nan,nan,nan,743.37518,1.6544337e-03,1030.75434,1.5672805e-03,4.4078931e-02,8.4882207e-02


### Retrieve periodogram information

In [22]:
read_hdul['GLS_PG']

In [23]:
read_hdul['GLS_PG'].header

XTENSION= 'BINTABLE'           / binary table extension                         
BITPIX  =                    8 / array data type                                
NAXIS   =                    2 / number of array dimensions                     
NAXIS1  =                   16 / length of dimension 1                          
NAXIS2  =              1000001 / length of dimension 2                          
PCOUNT  =                    0 / number of group parameters                     
GCOUNT  =                    1 / number of groups                               
TFIELDS =                    2 / number of table fields                         
TTYPE1  = 'freq    '                                                            
TFORM1  = 'D       '                                                            
TUNIT1  = 'd-1     '                                                            
TTYPE2  = 'power   '                                                            
TFORM2  = 'D       '        

In [24]:
read_hdul['GLS_PG'].data[:10]

FITS_rec([(4.8      , 0.00024938), (4.8001392, 0.00024974),
          (4.8002784, 0.00025008), (4.8004176, 0.00025039),
          (4.8005568, 0.00025069), (4.800696 , 0.00025097),
          (4.8008352, 0.00025122), (4.8009744, 0.00025146),
          (4.8011136, 0.00025167), (4.8012528, 0.00025186)],
         dtype=(numpy.record, [('freq', '>f8'), ('power', '>f8')]))

In [25]:
read_pg_table = QTable.read(read_hdul['GLS_PG'])
read_pg_table

freq,power
1 / d,
float64,float64
4.8,0.0002493765538039762
4.800139199999999,0.00024973683565817766
4.8002784,0.0002500762492685541
4.800417599999999,0.0002503947399286319
4.8005568,0.00025069225989698
4.800695999999999,0.00025096876841448684
4.8008352,0.00025122423171862506
4.800974399999999,0.0002514586230550532


In [26]:
read_pg_table['freq'][:-1]

<Quantity [  4.8      ,   4.8001392,   4.8002784, ..., 143.9995824,
           143.9997216, 143.9998608] 1 / d>

In [27]:
diff_frecs = read_pg_table['freq'][:-1] - read_pg_table['freq'][1:]
diff_frecs

<Quantity [-0.0001392, -0.0001392, -0.0001392, ..., -0.0001392, -0.0001392,
           -0.0001392] 1 / d>

### Delete read information

In [28]:
read_hdul.close()

In [29]:
del read_hdul

## Check final results that yielded error

In [30]:
invalid_tess_pg = gto.loc[gto['valid_PG_TESS']==0, ['Karmn', 'error_PG_TESS']]
invalid_tess_pg

,Karmn,error_PG_TESS
0,J23585+076,Not recognized as a supported data product:\nn...
1,J23556-061,Not recognized as a supported data product:\nn...
4,J23492+024,Not recognized as a supported data product:\nn...
6,J23419+441,Not recognized as a supported data product:\nn...
8,J23351-023,Not recognized as a supported data product:\nn...
...,...,...
373,J01056+284,Not recognized as a supported data product:\nn...
384,J00184+440,Not recognized as a supported data product:\nn...
386,J00162+198W,Not recognized as a supported data product:\nn...
387,J00162+198E,Not recognized as a supported data product:\nn...


In [31]:
invalid_tess_pg['error_PG_TESS'] = invalid_tess_pg['error_PG_TESS'].astype(str)

In [32]:
invalid_tess_pg.groupby(by=['error_PG_TESS']).count()

,Karmn
error_PG_TESS,
Floating point nan values are not allowed in FITS headers.,5
Not recognized as a supported data product:\nnan\nThis file may be corrupt due to an interrupted download. Please remove it from your disk and try again.,119


**Observation:** We have: 
- 5 objects for which the _TESS_ _FITS_ header is not correctly identifiable.
- 0 objects for which there seems to be more than one _TESS_ file under the relevant data folder, but...
  - **IMPORTANT NOTE:** two approaches have been used for the objects that have more than one file available:
    - For the _TESS_ lightcurves already proveded initially: the first file under the folder is selected (sorted by sector name, so that the sector with the higher number is selected).
    - For the additional _TESS_ light curves downloaded from _MAST_ server, teh light curve with the larger total exposition time was selected.
- 123 objects for which there is no TESS file available.

These are the 5 objects for which the _FITS_ header seems incorrect. We will try to open them with another, alternative program.

In [33]:
invalid_tess_pg[invalid_tess_pg['error_PG_TESS'].str.contains("Floating")]['Karmn']

49     J20450+444
62     J19251+283
63     J19242+755
227    J10125+570
243    J09307+003
Name: Karmn, dtype: object

**NO OBJECTS FOUND WITH THIS ERROR:** These are the \*9\* objects for which the _TESS_ folder seems to have more than one file. We will look individually to each one of them.

In [34]:
invalid_tess_pg[invalid_tess_pg['error_PG_TESS'].str.contains("ambiguous")]['Karmn']

Series([], Name: Karmn, dtype: object)

In [35]:
mask = invalid_tess_pg['error_PG_TESS'].str.contains("ambiguous") | \
    invalid_tess_pg['error_PG_TESS'].str.contains("FITS")
objects_to_check = invalid_tess_pg[mask][['Karmn', 'error_PG_TESS']] \
    .sort_values(by=['error_PG_TESS', 'Karmn']) \
    .reset_index(drop=True)
objects_to_check

,Karmn,error_PG_TESS
0,J09307+003,Floating point nan values are not allowed in F...
1,J10125+570,Floating point nan values are not allowed in F...
2,J19242+755,Floating point nan values are not allowed in F...
3,J19251+283,Floating point nan values are not allowed in F...
4,J20450+444,Floating point nan values are not allowed in F...


In [36]:
objects_to_check.to_csv(TESS_EXCEPTION_FILE, sep=',', decimal='.', index=False)

# Summary

**OBSERVATIONS AND CONCLUSIONS:**
- We have completed the basic GLS periodogram calculation for the available TESS light curves for CARM_VIS objetcs, and stored the results.
- Some of the objects yielded errors, and some others yielded warnings. Those objects should be analized individually.
  - Errors in _TESS_ file _FITS_ header:
    - J20450+444
    - J19251+283
    - J19242+755
    - J10125+570
    - J09307+003
  - No objects were left with several possible, available files, after selecting the first available file, or the downloaded file with the larget total exposition time.
